### Installing hdf5 (for iPic3D) on Leonardo without web access on the machine

0. **Connect** shared _leo_folder_ to Leonardo and login in it  
 > sshfs leonardo:/leonardo/home/userinternal/aromeo00/ leo_folder/  
 > ssh leonardo  
 * **IMPORTANT**: remember to unmount it after usage!!!  
   > sudo umount -f leo_folder  

1. At the beginning one should **source** a simple bash script to load the already present version of spack
 _in start.sh_ > source /home/cinprod/spack_setup.sh   
               > module use /home/cinprod/spack/02/modules/BA/0.19  
               > module load spack
 * check if everything is ok by invoking 
   > spack find  
 > chmod +x start.sh  
 > source start.sh  

2. **Create** a new directory for the project and useful folders there inside
> mkdir iPic3D_spack  
> cd iPic3D_spack  
> mkdir envs mirror recipies source_cache  
> cd envs  
> mkdir nvhpc-2023  

3. **Create** a new environment inside the last folder
> cd nvhpc-2023  
> spack env create -d .  
 * This generates a _spack.yaml_ file that needs to be modified by coping lines present in _spack.yaml_ provided by Nitin by deleting from _specs_ the dependencies from petsc and ecsim
 
4. **Activate** the environment
> spack env activate -d envs/nvhpc-2023/  
 * _spack env status_ will display environment status
 
5. **Add** a spack repository in recipies
 > cd recipies  
 > spack repo add --scope env:/leonardo/home/userinternal/aromeo00/iPic3D_spack/envs/nvhpc-2023 /leonardo/home/userinternal/aromeo00/iPic3D_spack/recipies/ 
  * This will create a file _repo.yaml_ with the namespace for the project that should be _repo_

6. **Create** package directory in repo folder and copy hdf5 folder from Leonardo repo
 > mkdir package  
 > cp -r /home/cinprod/spack/02/spack_sources/0.19.1_preprod_base/var/spack/repos/builtin/packages/hdf5 .  

7. **Load** and **check** nvhpc compiler
 > module load nvhpc/23.1-x5l  
 > spack compiler find  
  * This shows where compilers.yaml which can be open (/home/cinprod/spack/02/spack_sources/0.19.1_preprod_base/etc/spack/compilers.yaml) to see compiler specifications and flags
 > spack compilers  
  * This shows the list of available compilers in current environment

8. **Copy** in _mirror_ folder the source for desired version of hdf5 library in tar format (downloaded from official website, in this case 1.10.6)
 > cd ../../../mirror  
 > mkdir hdf5  
 _through local terminal_ > cp -r ../Downloads/hdf5-1.10.6.tar.gz _in mirror/hdf5_  

 9. **Modify** the following files:
  - _spack.yaml_ : in repos there should be the correct directory
   > /leonardo/home/userinternal/aromeo00/iPic3D_spack/recipies  
  - _recipies/ipic3d/package/packages.py_: comment current url and put source link to hdf5 tar (NB: this file can be accessed by typing _spack edit hdf5_)
   > url = "/leonardo/home/userinternal/aromeo00/iPic3D_spack/mirror/hdf5/hdf5-1.10.10.tar.gz"   
  - moreover add in the same file the following variant:  
   > variant("parallel", default=False, description="Enable the parallel")  

 10. Type _spack concretize -f_ or _spack spec -Il hdf5_
 
 11. **spack install**
 
 12. In order to check hdf5 installed versions type
     > spack find -lvp hdf5@1.10.6%nvhpc




### iPic3D on Leonardo
spack create ipic3d  
gitcloning folder and tar project from https://github.com/CmPA/iPic3D.git in local, then move it on a specific folder on _mirror_   
spack edit ipic files (and hdf5?) to put correct links to project  
spack install  

error: problem in running on Leo compute nodes
> [aromeo00@login14 ~]$ srun -N 1 --ntasks-per-node=4 -p prod -t 2:00:00 --pty /bin/bash  
> srun: error: Unable to allocate resources: Invalid account or account/partition combination specified

inside mirror/iPIC3D/iPic3D ---> mkdir build  
ml  
spack module tcl refresh   
module load hdf5/1.10.6--openmpi--4.1.4--nvhpc--23.1-hbg  
module load cmake
cmake ..
ccmake .. (configuration file c - e - g)  
cmake ..  

make  

mpirun -n 4 ../build/iPic3D GEM.inp  


### iPic3D on M100

spenv  
module load autoload hdf5/1.12.0--spectrum_mpi--10.3.1--binary  
module load cmake  
git clone https://github.com/CmPA/iPic3D.git
cd iPic3D  
mkdir build  
cmake ..  
ccmake .. (for setting configuration)  
cmake ..  
make -j (for parallel compilation, -B for serial and autoclean)    

Requesting a compute node to execute the job  
> salloc --account cin_staff --partition m100_usr_prod --qos m100_qos_dbg --time 01:00:00 --nodes 1 --ntasks-per-node=4 --job-name=IPIC3D --exclusive

NB: if srun is used to allocate the job for some reason one needs --cpus-per-task>=4 (and --pty /bin/bash)

Execution
> mpirun -n 4 ./iPic3D ../inputfiles/GEM.inp

Errors:
1. problem in recognizing pthread stuff, solved by adding in CMakeList "-lpthread" compilation flag
> set (CMAKE_CXX_FLAGS " -lpthread -g ")  
2. hdf5/1.12.0--spectrum_mpi--10.4.0-binary does not work  
3. something wrong happens when he's trying to open/write HDF5 files:
> HDF5-DIAG: Error detected in HDF5 (1.12.0) MPI-process 2:  
> #000: H5F.c line 705 in H5Fcreate(): unable to create file  
>    major: File accessibility  
>    minor: Unable to open file

    __Solution: create data directory in build__
    


### iPic3D on Karolina

module load HDF5/1.10.7-gompi-2021a  
module load CMake/3.20.1-GCCcore-10.3.0  
git clone https://github.com/CmPA/iPic3D.git  
cd iPic3D  
mkdir build  

in CMakeList.txt put the subsequent line to avoid pthread library errors:
> set (CMAKE_CXX_FLAGS " -lpthread -g ") 

cmake ..  
ccmake .. (for setting configuration)  
cmake ..  
make -j (for parallel compilation, -B for serial and autoclean)   

In order to have a folder for output files
> mkdir data

Create a pbs script to run the job
> #!/bin/bash  
> #PBS -A DD-23-44  
> #PBS -q qgpu  
> #PBS -l select=1:mpiprocs=4  
> #PBS -l walltime=08:00:00  
> #PBS -N IPIC3D  
> #PBS -m n  
> #PBS -M a.romeo@cineca.it  
> 
> module purge  
> module load HDF5/1.10.7-gompi-2021a  
> module list  
> 
> nvidia-smi  
> 
> cd iPic3D/build  
> mpirun -n 4 ./iPic3D ../inputfiles/GEM.inp  

Errors:
1. problems in reading correctly some input quantity with HDF5/1.12.2-gompi-2022a loaded
    > *** MAXWELL SOLVER ***  
    > Initial residual: -nan norm b vector (source) = -nan
